In [135]:
import numpy as np
from scipy.io import loadmat
import open3d
import scipy
from scipy.spatial import distance

def rigid_motion(p,q):
    """
    Least-Squares Rigid Motion Using Singular Value Decomposition. 
    (https://igl.ethz.ch/projects/ARAP/svd_rot.pdf) 
    
    (note: so far only for the easy case, where all weights are = 1)
    
    p,q: shape [num_points, 3]
    
    """
    n,d = p.shape
    
    # compute centroids
    p_cen = sum(p)/len(p)
    q_cen = sum(q)/len(q)
    
    # compute centered vectors
    X = np.array([i-p_cen for i in p])
    Y = np.array([i-q_cen for i in q])
    
    # compute covariance matrix 
    W = np.eye(n)
    S =  X.T.dot(W).dot(Y)
    
    # compute sigular value decomposition
    U, _, V = np.linalg.svd(S)
    
    # compute optimal R and t
    M = np.eye(d)
    M[-1,-1] = np.linalg.det(V.dot(U.T))
    R = V.dot(M).dot(U.T)
    
    t = q_cen - R.dot(p_cen)
    
    return R, t
    

In [102]:
# print("Load a ply point cloud, print it, and render it")
# pcd1  = open3d.read_point_cloud("Data/data/0000000000.pcd")
# pcd2  = open3d.read_point_cloud("Data/data/0000000001.pcd")
# print(np.asarray(pcd.points))
# open3d.draw_geometries([pcd])
a_1 = loadmat("Data/source.mat")["source"].T
a_2 = loadmat("Data/target.mat")["target"].T

In [ ]:
def rms_error(p, q):
    # TODO: This error calculation right?
    n = p.shape[0]
    dist = [distance.euclidean(p[i,:], q[i,:]) for i in range(n)]
    return sum(np.power(dist, 2))/n

def icd(a_1, a_2, max_it=20):
    """
    a_1: positions of points in point cloud 1. shape : [num_points1, 3]
    a_2: positions of points in point cloud 2. shape : [num_points2, 3]
    
    """
    n,d = a_1.shape
    
    # Filter the point clouds based on the depth,
    # only keep the indices where the z of the point cloud is less than 1
    valid_bool_1 = (a_1[:,2])<1
    valid_bool_2 = (a_2[:,2])<1
    a_1 = a_1[valid_bool_1]
    a_2 = a_2[valid_bool_2]
    
    R_overall = np.eye(d)
    t_overall = np.zeros(d)
    
    # TODO: Base loop on difference in rsm error
    #rms_error_old = 999999999
    #rms_error_new = rms_error_old-1
    for _ in range(max_it):
            
        # (Step 1) Find closest points for each point in a_1 from a_2
        tree = scipy.spatial.KDTree(a_2)
        closest_dists, closest_idx = tree.query(a_1)
        # Found this on stackoverflow: https://bit.ly/2P8IYiw
        # Not sure if we can use this, but it is definetely much (!!) faster 
        # than manually comparing all the vectors.
        # Usage also proposed on Wikipedia: https://bit.ly/2urg9nU
        # For how-to-use see: https://bit.ly/2UbKNfn
        closest_a_2 = a_2[closest_idx]
    
        # (Step 2) Refine R and t using Singular Value Decomposition
        R, t = rigid_motion(a_1,closest_a_2)
       
        # update a_1
        a_1 = np.array([R.dot(a)+t for a in a_1])
        
        # update rms error
        # rms_error_old = rms_error_new
        rms_error_new = rms_error(a_1, closest_a_2)
        print(rms_error_new)
        
        # update overall R and t
        # R_overall = R.dot(R_overall)
        # t_overall = R.dot(t_overall) + t
        
icd(a_1, a_2)

7.847178869938536
6.034980464688789
6.243586654593772
5.995267518771472
5.979234335289667
5.490952687967517
5.064767260627452
3.9096597718726533
2.687111657152096
